# Path setup

In [36]:
main_path = '../Data/videos/'

# Importations and Installations

In [37]:
import json
import os

import cv2
import mediapipe as mp
from mediapipe.framework.formats import landmark_pb2
import numpy as np
import pandas as pd

# Imports
import os

from sklearn.linear_model    import LinearRegression

from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore') # ignore some warnings


SEED = 123456789   # Define a random seed so random steps can be replicated. 


In [38]:
left_hand = ["LEFT_THUMB.x", "LEFT_INDEX.x", "LEFT_PINKY.x",  "LEFT_WRIST.x", "LEFT_THUMB.y", "LEFT_INDEX.y", "LEFT_PINKY.y",  "LEFT_WRIST.y", "LEFT_THUMB.v", "LEFT_INDEX.v", "LEFT_PINKY.v",  "LEFT_WRIST.v"]
right_hand = ["RIGHT_THUMB.x", "RIGHT_INDEX.x", "RIGHT_PINKY.x",  "RIGHT_WRIST.x", "RIGHT_THUMB.y", "RIGHT_INDEX.y", "RIGHT_PINKY.y",  "RIGHT_WRIST.y", "RIGHT_THUMB.v", "RIGHT_INDEX.v", "RIGHT_PINKY.v",  "RIGHT_WRIST.v"]
left_foot = ["LEFT_FOOT_INDEX.x", "LEFT_ANKLE.x", "LEFT_HEEL.x", "LEFT_FOOT_INDEX.y", "LEFT_ANKLE.y", "LEFT_HEEL.y", "LEFT_FOOT_INDEX.v", "LEFT_ANKLE.v", "LEFT_HEEL.v"]
right_foot = ["RIGHT_FOOT_INDEX.x", "RIGHT_ANKLE.x", "RIGHT_HEEL.x", "RIGHT_FOOT_INDEX.y", "RIGHT_ANKLE.y", "RIGHT_HEEL.y", "RIGHT_FOOT_INDEX.v", "RIGHT_ANKLE.v", "RIGHT_HEEL.v"]
extremities = {"left_hand" : left_hand, "right_hand": right_hand, "left_foot" : left_foot, "right_foot" : right_foot}

# Train model to predict position of the body landmarks from extremities

### First get pose sequence information to train the model to predict the landmarks position

In [39]:
videos = ['boulder_1_04', 'boulder_2_02', 'boulder_3_03']

df_final = pd.DataFrame()
dfs = []
for video in videos:
    data =json.load(open(main_path + video + '/' + video + '_POSE.json'))
    df = pd.json_normalize(data)
    dfs.append(df)


In [40]:
# Concatenate the dataframe into a unique one to use as a train/test set
df_final = pd.concat(dfs)
df_final

,LEFT_EYE_INNER.x,LEFT_EYE_INNER.y,LEFT_EYE_INNER.v,LEFT_EYE.x,LEFT_EYE.y,LEFT_EYE.v,LEFT_EYE_OUTER.x,LEFT_EYE_OUTER.y,LEFT_EYE_OUTER.v,RIGHT_EYE_INNER.x,...,RIGHT_HIP.v,LEFT_HIP.x,LEFT_HIP.y,LEFT_HIP.v,RIGHT_SHOULDER.x,RIGHT_SHOULDER.y,RIGHT_SHOULDER.v,LEFT_SHOULDER.x,LEFT_SHOULDER.y,LEFT_SHOULDER.v
0,0.574033,0.632629,0.999316,0.572735,0.632456,0.999181,0.571453,0.632280,0.999160,0.577669,...,0.999895,0.591523,0.607604,0.999904,0.598253,0.634587,0.999605,0.569853,0.629704,0.999765
1,0.601052,0.621455,0.999380,0.602316,0.621491,0.999259,0.603565,0.621529,0.999236,0.596943,...,0.999874,0.596516,0.605089,0.999874,0.576945,0.616103,0.999619,0.613640,0.618872,0.999763
2,0.589538,0.629733,0.999425,0.587694,0.629856,0.999308,0.585846,0.629968,0.999296,0.594478,...,0.999847,0.587995,0.595155,0.999840,0.613829,0.623784,0.999636,0.583776,0.627473,0.999769
3,0.629600,0.612378,0.999426,0.630356,0.612422,0.999278,0.630382,0.612509,0.999297,0.629757,...,0.999772,0.624270,0.663279,0.999854,0.620764,0.625918,0.999407,0.638795,0.630043,0.999747
4,0.632866,0.612649,0.999261,0.633064,0.612679,0.999093,0.633111,0.612744,0.999110,0.633617,...,0.999745,0.612128,0.665145,0.999858,0.634944,0.626695,0.999319,0.633006,0.630054,0.999752
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1311,1.014892,0.929562,0.434201,1.015876,0.928105,0.426506,1.016831,0.926704,0.402676,1.011300,...,0.831660,0.987929,0.954359,0.778419,0.986780,0.897651,0.994460,1.025928,0.901987,0.849996
1312,1.000675,0.940998,0.431597,1.001510,0.939381,0.425284,1.002323,0.937803,0.405078,0.997467,...,0.848159,0.971342,0.950287,0.797535,0.967627,0.921801,0.994165,1.004301,0.897119,0.858585
1313,0.979813,0.941740,0.409418,0.980392,0.940917,0.402969,0.980962,0.940150,0.382002,0.978663,...,0.863060,0.962534,0.960960,0.816755,0.968952,0.929526,0.993315,0.988434,0.918927,0.861747
1314,0.992065,0.942951,0.371301,0.992881,0.942466,0.365918,0.993722,0.942027,0.346639,0.989608,...,0.876467,0.973271,0.964853,0.833865,0.973639,0.918558,0.990020,1.000075,0.927716,0.850441


In [41]:
# Setup the input and output variables for the linear regression model:
list_extremities = left_hand + right_hand + left_foot + right_foot
list_body = list(set(list(df_final.columns)) - set(list_extremities))
list_extremities = [x for x in list_extremities if ".v" not in x]
list_body = [x for x in list_body if ".v" not in x]
list_visibilities = [x for x in list(df_final.columns) if ".v" in x]

X = pd.DataFrame(df_final, columns = list_extremities)
y = pd.DataFrame(df_final, columns = list_body) 


In [42]:
# Divide data into a training set and a testing set using the train_test_split() function
#   NOTE: Please call the subsamples `X_train` ,  `X_test` , `y_train` , `y_test` so we can follow your code
indices = range(0,X.shape[0],1)
X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(X, y, indices, test_size=0.20, random_state=SEED)

In [43]:
regr = LinearRegression()
  
regr.fit(X_train, y_train)
print(regr.score(X_test, y_test))

0.9912075105745203


We have a model trained to predict the landmarks of the body from the landmarks of the extremities. Now, we use it to generate the skeleton of the climber from the move sequence

First, test by reading the pose sequence of a climb, removing the body landmarks, and predicting them with the model

In [44]:
video = 'boulder_3_03'

data =json.load(open(main_path + video + '/' + video + '_POSE.json'))
df = pd.json_normalize(data)

In [45]:
X_pred = pd.DataFrame(df, columns = list_extremities)
y_true = pd.DataFrame(df_final, columns = list_body) 

y_pred = pd.DataFrame(regr.predict(X_pred), columns = list_body)

In [46]:
X_pred.dropna(inplace=True)
X_pred

,LEFT_THUMB.x,LEFT_INDEX.x,LEFT_PINKY.x,LEFT_WRIST.x,LEFT_THUMB.y,LEFT_INDEX.y,LEFT_PINKY.y,LEFT_WRIST.y,RIGHT_THUMB.x,RIGHT_INDEX.x,...,LEFT_HEEL.x,LEFT_FOOT_INDEX.y,LEFT_ANKLE.y,LEFT_HEEL.y,RIGHT_FOOT_INDEX.x,RIGHT_ANKLE.x,RIGHT_HEEL.x,RIGHT_FOOT_INDEX.y,RIGHT_ANKLE.y,RIGHT_HEEL.y
0,0.717822,0.715049,0.713584,0.719001,0.836007,0.834201,0.837324,0.841786,0.722705,0.721349,...,0.759021,0.987114,0.961926,0.962820,0.745860,0.743190,0.746888,0.969018,0.952316,0.954278
1,0.717146,0.714825,0.712414,0.718297,0.835169,0.833148,0.837048,0.841869,0.752909,0.753957,...,0.729580,0.992583,0.976974,0.975361,0.726144,0.728173,0.730390,0.976196,0.959083,0.964651
2,0.717489,0.715424,0.712459,0.718279,0.834538,0.829193,0.836460,0.841839,0.759042,0.760424,...,0.738480,0.984117,0.963582,0.962091,0.739003,0.738240,0.740984,0.975618,0.957984,0.963899
3,0.718114,0.715927,0.713132,0.719119,0.834541,0.830753,0.836458,0.841794,0.746055,0.746317,...,0.734322,0.970710,0.950739,0.954490,0.728838,0.729971,0.732119,0.985690,0.966255,0.973099
4,0.718560,0.716166,0.714003,0.719557,0.834551,0.830001,0.836554,0.841912,0.743062,0.742921,...,0.736717,1.001474,0.978452,0.982121,0.753084,0.751091,0.754938,0.983900,0.959323,0.964579
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1311,0.992229,0.993439,0.997801,0.995960,0.964092,0.965259,0.963996,0.961998,0.962713,0.961442,...,0.944060,0.984168,0.971874,0.967676,0.935223,0.935174,0.936426,0.986195,0.969109,0.963876
1312,1.005624,1.010446,1.015085,1.005982,0.961154,0.961283,0.958877,0.957802,0.960460,0.959331,...,0.944251,0.982160,0.972687,0.970284,0.935532,0.934570,0.934621,0.984078,0.975712,0.971312
1313,0.985027,0.986787,0.987460,0.983128,0.961408,0.962587,0.959168,0.960660,0.957089,0.957016,...,0.930413,0.984032,0.976297,0.976504,0.925293,0.933911,0.932135,0.988396,0.979391,0.976724
1314,0.969721,0.969177,0.970202,0.972743,0.971176,0.974075,0.974893,0.971355,0.958161,0.956227,...,0.937920,0.986380,0.977219,0.975094,0.922298,0.933869,0.934225,0.989029,0.977539,0.977355


In [47]:
y_pred

,LEFT_ELBOW.y,RIGHT_EYE_INNER.x,NOSE.x,NOSE.y,LEFT_EAR.x,RIGHT_KNEE.x,MOUTH_LEFT.y,LEFT_KNEE.x,LEFT_EYE_INNER.y,RIGHT_ELBOW.y,...,LEFT_HIP.x,MOUTH_LEFT.x,LEFT_EYE_INNER.x,MOUTH_RIGHT.x,LEFT_EAR.y,RIGHT_EYE_INNER.y,RIGHT_ELBOW.x,LEFT_SHOULDER.x,LEFT_ELBOW.x,RIGHT_EYE.x
0,0.854658,0.740810,0.739684,0.808836,0.740775,0.737746,0.813020,0.723999,0.804586,0.845381,...,0.735896,0.739535,0.739880,0.740412,0.806674,0.804519,0.740535,0.738082,0.731486,0.741200
1,0.856775,0.744650,0.742786,0.807561,0.744765,0.730767,0.811961,0.714786,0.802834,0.851173,...,0.733240,0.742515,0.743609,0.743422,0.804566,0.802990,0.753499,0.740115,0.730948,0.745127
2,0.859871,0.743585,0.742028,0.807983,0.743678,0.734900,0.811930,0.707865,0.803180,0.852217,...,0.731032,0.742096,0.742377,0.743190,0.804357,0.803567,0.757212,0.739284,0.731294,0.744113
3,0.855954,0.747522,0.745887,0.799210,0.750053,0.731072,0.803666,0.718170,0.793963,0.845200,...,0.736765,0.746512,0.747107,0.746773,0.795159,0.794150,0.752754,0.748236,0.735591,0.747841
4,0.855335,0.741260,0.739753,0.792385,0.742108,0.742060,0.796658,0.703858,0.786809,0.839534,...,0.733028,0.740214,0.740141,0.741230,0.787989,0.787192,0.752248,0.739959,0.733540,0.741790
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1311,0.940407,0.986698,0.985023,0.889252,0.991496,0.943792,0.893396,0.970505,0.884931,0.921841,...,0.973836,0.985053,0.988101,0.983228,0.885743,0.883852,0.949096,0.994687,1.002731,0.986173
1312,0.943763,0.973336,0.972095,0.911038,0.975484,0.940276,0.914632,0.959365,0.907608,0.939849,...,0.958469,0.971134,0.974401,0.969871,0.908881,0.906665,0.943063,0.975500,0.993646,0.972759
1313,0.956547,0.982550,0.980048,0.923912,0.985515,0.946771,0.927490,0.944663,0.920725,0.962397,...,0.962215,0.979200,0.983231,0.978274,0.922192,0.920073,0.946452,0.981434,0.979509,0.982295
1314,0.948334,0.987392,0.985499,0.887493,0.992436,0.947401,0.891235,0.960005,0.883514,0.933426,...,0.979988,0.986150,0.988427,0.984719,0.883910,0.882495,0.959128,0.994363,0.988800,0.987162


In [48]:
# We now have a dataframe <df_pred> containing all the spatial informations of the landmarks at each frame
df_pred = pd.concat([X_pred, y_pred], axis=1)

# Append the visibilities (by default = 1) to have complete informations for the landmarks
for v in list_visibilities:
    df_pred[v] = 1

   
df_pred = df_pred.reindex(sorted(df.columns), axis=1)
df_pred

,LEFT_ANKLE.v,LEFT_ANKLE.x,LEFT_ANKLE.y,LEFT_EAR.v,LEFT_EAR.x,LEFT_EAR.y,LEFT_ELBOW.v,LEFT_ELBOW.x,LEFT_ELBOW.y,LEFT_EYE.v,...,RIGHT_PINKY.y,RIGHT_SHOULDER.v,RIGHT_SHOULDER.x,RIGHT_SHOULDER.y,RIGHT_THUMB.v,RIGHT_THUMB.x,RIGHT_THUMB.y,RIGHT_WRIST.v,RIGHT_WRIST.x,RIGHT_WRIST.y
0,1,0.751496,0.961926,1,0.740775,0.806674,1,0.731486,0.854658,1,...,0.830534,1,0.748799,0.826202,1,0.722705,0.829116,1,0.726399,0.834478
1,1,0.724282,0.976974,1,0.744765,0.804566,1,0.730948,0.856775,1,...,0.832009,1,0.754268,0.829084,1,0.752909,0.828983,1,0.754003,0.834769
2,1,0.728468,0.963582,1,0.743678,0.804357,1,0.731294,0.859871,1,...,0.831063,1,0.755474,0.829000,1,0.759042,0.827010,1,0.759749,0.833472
3,1,0.729509,0.950739,1,0.750053,0.795159,1,0.735591,0.855954,1,...,0.829987,1,0.757826,0.819754,1,0.746055,0.826689,1,0.747694,0.832488
4,1,0.726453,0.978452,1,0.742108,0.787989,1,0.733540,0.855335,1,...,0.823240,1,0.753784,0.812431,1,0.743062,0.821413,1,0.745446,0.826680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1311,1,0.947350,0.971874,1,0.991496,0.885743,1,1.002731,0.940407,1,...,0.975079,1,0.973287,0.896452,1,0.962713,0.973770,1,0.956895,0.961696
1312,1,0.945864,0.972687,1,0.975484,0.908881,1,0.993646,0.943763,1,...,0.978422,1,0.958798,0.920617,1,0.960460,0.977680,1,0.955508,0.969207
1313,1,0.932187,0.976297,1,0.985515,0.922192,1,0.979509,0.956547,1,...,0.984347,1,0.972107,0.936519,1,0.957089,0.981523,1,0.952559,0.978151
1314,1,0.938594,0.977219,1,0.992436,0.883910,1,0.988800,0.948334,1,...,0.987196,1,0.982176,0.895699,1,0.958161,0.983290,1,0.955772,0.972139


Finally, use Mediapipe to visualize the generated skeleton

In [49]:
# Mediapipe parameters
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils 
mp_drawing_styles = mp.solutions.drawing_styles

In [50]:
# Setup parameters to prepare the video
ordered_columns = [ 'NOSE', 'LEFT_EYE_INNER', 'LEFT_EYE', 'LEFT_EYE_OUTER', 'RIGHT_EYE_INNER', 'RIGHT_EYE', 'RIGHT_EYE_OUTER',
                    'LEFT_EAR', 'RIGHT_EAR', 'MOUTH_LEFT', 'MOUTH_RIGHT', 'LEFT_SHOULDER', 'RIGHT_SHOULDER',
                    'LEFT_ELBOW', 'RIGHT_ELBOW', 'LEFT_WRIST', 'RIGHT_WRIST', 'LEFT_PINKY', 'RIGHT_PINKY',
                    'LEFT_INDEX', 'RIGHT_INDEX', 'LEFT_THUMB', 'RIGHT_THUMB', 'LEFT_HIP', 'RIGHT_HIP', 'LEFT_KNEE',
                    'RIGHT_KNEE', 'LEFT_ANKLE', 'RIGHT_ANKLE', 'LEFT_HEEL', 'RIGHT_HEEL', 'LEFT_FOOT_INDEX',
                    'RIGHT_FOOT_INDEX']

def draw_predicted_skeleton(video, df_pred, frame_step = 10, drawing_with_MP = True, extension = '_PRED.mp4'):

    directory = main_path + video + '/'
    img = cv2.imread(directory + video + '_SCREEN.jpg')

    frame_width = img.shape[1]
    frame_height = img.shape[0]

    fps = 60

    # Outpath
    out = cv2.VideoWriter(directory + video  + extension, cv2.VideoWriter_fourcc(*'MP4V'), fps, (frame_width, frame_height))

    # Loop over all landmarks and draw it on the video based on its informations
    
    for i in range(df_pred.shape[0]): # Loop over the frames
        if i % frame_step != 0: continue

        if i % 100 == 0: print(f"Processing frame {i}")
        
        img = cv2.imread(directory + video + '_SCREEN.jpg')

        if (drawing_with_MP):
            # Convert the predicted informations to a format that can be read as results.pose_landmarks

            landmarks_list = []

            for l in ordered_columns:
                x = df_pred[l+'.x'][i] 
                y = df_pred[l+'.y'][i] 
                z = 0
                v = df_pred[l+'.v'][i] 

                landmarks_list.append({'x' : x, 'y' : y, 'z' : z, 'visibility' : v})
            
            pose_landmarks = landmark_pb2.NormalizedLandmarkList(
                landmark = landmarks_list
            )


            # Draw the landmarks on the video
            mp_drawing.draw_landmarks(
                img,
                pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                landmark_drawing_spec = mp_drawing.DrawingSpec(color=(245,117,66), thickness=3, circle_radius=2),
                connection_drawing_spec = mp_drawing.DrawingSpec(color=(245,66,230), thickness=3, circle_radius=2))
                # landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())

        else :
            for j in range(0, df_pred.shape[1], 3):
                x = df_pred.iloc[i,j+1] 
                y = df_pred.iloc[i,j+2]
                img = cv2.circle(img, (int(x*frame_width), int(y*frame_height)), radius=10, color=(255,0,0), thickness=-1)
        
        out.write(img)

    out.release()
       


In [51]:
# draw_predicted_skeleton(video, df_pred=df_pred, frame_step = 5, drawing_with_MP=True)

# Now, generate sequence of landmarks from the move sequence

Instead of generating the skeleton from the landmarks, first generate the landmarks from the move sequence, and then generate the skeleton from the landmarks

### First, we do it by using frames as references [OBSOLETE]

In [52]:
# Recover the move sequence from the csv file generated by the other Notebook
# directory = main_path + video + '/'
# df_sequence = pd.read_csv(main_path+ '/' + 'moonboard' + '/' + 'move_sequence.csv')
# df_sequence = df_sequence.iloc[:,1:]
# df_sequence

In [53]:
left_hand = ["LEFT_THUMB.x", "LEFT_INDEX.x", "LEFT_PINKY.x",  "LEFT_WRIST.x", "LEFT_THUMB.y", "LEFT_INDEX.y", "LEFT_PINKY.y",  "LEFT_WRIST.y", "LEFT_THUMB.v", "LEFT_INDEX.v", "LEFT_PINKY.v",  "LEFT_WRIST.v"]
right_hand = ["RIGHT_THUMB.x", "RIGHT_INDEX.x", "RIGHT_PINKY.x",  "RIGHT_WRIST.x", "RIGHT_THUMB.y", "RIGHT_INDEX.y", "RIGHT_PINKY.y",  "RIGHT_WRIST.y", "RIGHT_THUMB.v", "RIGHT_INDEX.v", "RIGHT_PINKY.v",  "RIGHT_WRIST.v"]
left_foot = ["LEFT_FOOT_INDEX.x", "LEFT_ANKLE.x", "LEFT_HEEL.x", "LEFT_FOOT_INDEX.y", "LEFT_ANKLE.y", "LEFT_HEEL.y", "LEFT_FOOT_INDEX.v", "LEFT_ANKLE.v", "LEFT_HEEL.v"]
right_foot = ["RIGHT_FOOT_INDEX.x", "RIGHT_ANKLE.x", "RIGHT_HEEL.x", "RIGHT_FOOT_INDEX.y", "RIGHT_ANKLE.y", "RIGHT_HEEL.y", "RIGHT_FOOT_INDEX.v", "RIGHT_ANKLE.v", "RIGHT_HEEL.v"]
extremities = {"left_hand" : left_hand, "right_hand": right_hand, "left_foot" : left_foot, "right_foot" : right_foot}

def generate_skeleton_from_move_seq(df_sequence, nb_frames):
    """ Generates the skeleton by interpolating between frames. This is not ideal
    since we want to generate it from just the holds only"""
    
    initial_nb_frames = df_sequence.iloc[-1]["frame"]
    scaling_factor = nb_frames/initial_nb_frames

    X_gen = pd.DataFrame(columns=list_extremities)
    last_frames = {"left_hand" : 0, "right_hand": 0, "left_foot" : 0, "right_foot" : 0}

    current_frame = 0
    # Initialize first position (which should be given by the first 4 frames for each extermity)
    X_gen.loc[0] = 0
    for i in range(4):
        key = df_sequence.iloc[i]["limb"]
        current_frame = df_sequence.iloc[i]["frame"]
        last_frames[key] = current_frame

        for extremity in extremities[key]:
            if(".x" in extremity):
                X_gen.iloc[0][extremity] = df_sequence.iloc[i]["x"]
            if(".y" in extremity):
                X_gen.iloc[0][extremity] = df_sequence.iloc[i]["y"]

    # Initialize all the frames from 0 to the current_frame in the df to
    # have the same sequence
    for i in range(current_frame+1):
        X_gen.loc[i] = X_gen.loc[0]

    # In case the first 4 frames are not the ones we want, initialize the missing values
    # with the mean of the coordinates 
    X_gen.fillna(X_gen.mean())

    # Now, go through the data frame and update lerp the coordinates of the limbs as they move
    
    for i in range(4, df_sequence.shape[0]):
        df_new = df_sequence.loc[i]
        # Get the current limb
        key = df_new["limb"]

        # Recover the last frame we saw it, and upadte it in the list
        old_frame = last_frames[key]
        new_frame = df_new["frame"]


        
        last_frames[key] = new_frame

        df_old = df_sequence.loc[df_sequence["frame"] == old_frame]
        df_old = df_old.loc[df_old["limb"] == key]
        
        # if df_old.empty:
        #     df_old = df_sequence.iloc[0]
        # else:
        #     df_old = df_old.iloc[0]


        # Compute the difference between the 2 frames to get an idea of the speed of motion      

        diff = new_frame - old_frame
        

        # Adjust the difference so that we have the required number of frames in the end
        new_diff = int(diff * scaling_factor)

        # # Interpolate the new coordinates over <new_diff> frames for all the extremities
        # # corresponding to the current limb
        for j in range(1,new_diff+1):
            current_frame += 1
            X_gen.loc[current_frame] = X_gen.loc[current_frame-1]


            coeff = j/new_diff
            for extremity in extremities[key]:
                if(".x" in extremity):
                    X_gen.iloc[current_frame][extremity] = (df_new["x"] - df_old["x"])*coeff + df_old["x"]
                if(".y" in extremity):
                    X_gen.iloc[current_frame][extremity] = (df_new["y"] - df_old["y"])*coeff + df_old["y"]



    return X_gen



In [54]:
# nb_frames = 500
# X_gen = generate_skeleton_from_move_seq(df_sequence, nb_frames)



# y_gen = pd.DataFrame(regr.predict(X_gen), columns = list_body)
# df_gen = pd.concat([X_gen, y_gen], axis=1)
# df_gen.to_csv(main_path + video + '/'+ video +'_generated_skeleton_frames.csv')

# for v in list_visibilities:
#     df_gen[v] = 1

# df_gen = df_gen.reindex(sorted(df.columns), axis=1)
# df_gen

In [55]:
# draw_predicted_skeleton(video, df_pred=df_gen, frame_step = 5, drawing_with_MP=True, extension = '_GEN_frames.mp4')

### Then, we do the same but without using the frame informations

In [56]:
main_path = '../Data/'
video = 'moonboard'

directory = main_path + video + '/'
df_sequence = pd.read_csv(directory + '/' + video + '_MOVE_SEQ.csv')
df_sequence = df_sequence.iloc[:,1:]
df_sequence

,x,y,limb
0,0.376667,0.733673,left_hand
1,0.583333,0.739796,right_hand
2,0.373333,0.931633,left_foot
3,0.611667,0.935714,right_foot
4,0.296667,0.532653,left_hand
5,0.593333,0.733673,right_foot
6,0.143333,0.691837,left_foot
7,0.608333,0.336735,right_hand
8,0.465000,0.334694,left_hand
9,0.683333,0.538776,right_foot


In [57]:
def generate_skeleton_from_move_seq2(df_sequence, nb_frames):
    """Generates the skeleton from the move sequence without using frame
    information. It therefore does a spatial interpolation"""

    # Compute scaling factor such that each move gets same number of frames (remove the first 4 because they're used to set the starting position)
    nb_frames_per_move = int(nb_frames/(df_sequence.shape[0]-4))


    X_gen = pd.DataFrame(columns=list_extremities, dtype=float)
    last_positions = {"left_hand" : [], "right_hand": [], "left_foot" : [], "right_foot" : []}


    current_frame = 0
    # Initialize first position (which should be given by the first 4 frames for each extermity)
    X_gen.loc[0] = 0.
    for i in range(4):
        key = df_sequence.iloc[i]["limb"]
        current_position = [df_sequence.iloc[i]["x"], df_sequence.iloc[i]["y"]]
        last_positions[key] = current_position

        for extremity in extremities[key]:
            if(".x" in extremity):
                X_gen.iloc[0][extremity] = float(df_sequence.iloc[i]["x"])
            if(".y" in extremity):
                X_gen.iloc[0][extremity] = df_sequence.iloc[i]["y"]


    # Initialize all the frames from 0 to the current_frame in the df to
    # have the same sequence
    for i in range(current_frame+1):
        X_gen.loc[i] = X_gen.loc[0]

    # In case the first 4 frames are not the ones we want, initialize the missing values
    # with the mean of the coordinates 
    X_gen.fillna(X_gen.mean())

    # Now, go through the data frame and update lerp the coordinates of the limbs as they move
    
    for i in range(4, df_sequence.shape[0]):
        df_new = df_sequence.loc[i]
        # Get the current limb
        key = df_new["limb"]

        # Recover the last frame we saw it, and upadte it in the list
        old_position= last_positions[key]
        new_position = [df_new["x"], df_new["y"]]

        last_positions[key] = new_position

        df_old = df_sequence.loc[(df_sequence["limb"] == key) & (df_sequence["x"] == old_position[0]) & (df_sequence["y"] == old_position[1])]
        # print(df_old)
        df_old = df_old.iloc[0]

        # if df_old.empty:
        #     df_old = df_sequence.iloc[0]
        # else:
        #     df_old = df_old.iloc[0]


        # Compute the distance between the 2 positions to get an idea of the speed of motion      

        dist = np.linalg.norm(np.array(new_position) - np.array(old_position))
        nb_frames_for_this_move = int(nb_frames_per_move * (dist/0.2))
        print(nb_frames_for_this_move)

        # # Adjust the difference so that we have the required number of frames in the end

        # # Interpolate the new coordinates over <new_diff> frames for all the extremities
        # # corresponding to the current limb
        for j in range(1,nb_frames_for_this_move+1):
            current_frame += 1
            X_gen.loc[current_frame] = X_gen.loc[current_frame-1]


            coeff = j/nb_frames_for_this_move
            for extremity in extremities[key]:
                if(".x" in extremity):
                    X_gen.iloc[current_frame][extremity] = (df_new["x"] - df_old["x"])*coeff + df_old["x"]
                if(".y" in extremity):
                    X_gen.iloc[current_frame][extremity] = (df_new["y"] - df_old["y"])*coeff + df_old["y"]


    return X_gen



In [58]:
nb_frames = 1500
X_gen = generate_skeleton_from_move_seq2(df_sequence, nb_frames)


y_gen = pd.DataFrame(regr.predict(X_gen), columns = list_body)
df_gen = pd.concat([X_gen, y_gen], axis=1)
df_gen.to_csv(main_path+ '/' + 'moonboard' + '/' + 'generated_skeleton.csv')

for v in list_visibilities:
    df_gen[v] = 1

df_gen = df_gen.reindex(sorted(df.columns), axis=1)
df_gen

135
126
207
252
162
134
137
190
162
119
124
351


,LEFT_ANKLE.v,LEFT_ANKLE.x,LEFT_ANKLE.y,LEFT_EAR.v,LEFT_EAR.x,LEFT_EAR.y,LEFT_ELBOW.v,LEFT_ELBOW.x,LEFT_ELBOW.y,LEFT_EYE.v,...,RIGHT_PINKY.y,RIGHT_SHOULDER.v,RIGHT_SHOULDER.x,RIGHT_SHOULDER.y,RIGHT_THUMB.v,RIGHT_THUMB.x,RIGHT_THUMB.y,RIGHT_WRIST.v,RIGHT_WRIST.x,RIGHT_WRIST.y
0,1,0.373333,0.931633,1,0.418326,0.692218,1,0.376901,0.739122,1,...,0.739796,1,0.486975,0.726139,1,0.583333,0.739796,1,0.583333,0.739796
1,1,0.373333,0.931633,1,0.418005,0.691575,1,0.376365,0.737804,1,...,0.739796,1,0.486740,0.725692,1,0.583333,0.739796,1,0.583333,0.739796
2,1,0.373333,0.931633,1,0.417683,0.690932,1,0.375830,0.736485,1,...,0.739796,1,0.486506,0.725244,1,0.583333,0.739796,1,0.583333,0.739796
3,1,0.373333,0.931633,1,0.417362,0.690290,1,0.375294,0.735167,1,...,0.739796,1,0.486271,0.724797,1,0.583333,0.739796,1,0.583333,0.739796
4,1,0.373333,0.931633,1,0.417040,0.689647,1,0.374758,0.733848,1,...,0.739796,1,0.486037,0.724350,1,0.583333,0.739796,1,0.583333,0.739796
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2095,1,0.545000,0.478571,1,0.729061,0.142984,1,0.736887,0.140991,1,...,0.128571,1,0.759805,0.174105,1,0.765000,0.128571,1,0.765000,0.128571
2096,1,0.545000,0.478571,1,0.729834,0.142615,1,0.737877,0.139977,1,...,0.128571,1,0.760294,0.173879,1,0.765000,0.128571,1,0.765000,0.128571
2097,1,0.545000,0.478571,1,0.730608,0.142247,1,0.738866,0.138963,1,...,0.128571,1,0.760782,0.173652,1,0.765000,0.128571,1,0.765000,0.128571
2098,1,0.545000,0.478571,1,0.731382,0.141879,1,0.739855,0.137949,1,...,0.128571,1,0.761270,0.173426,1,0.765000,0.128571,1,0.765000,0.128571


In [59]:
draw_predicted_skeleton(video, df_pred=df_gen, frame_step = 5, drawing_with_MP=True, extension = '_GEN_spatial.mp4')

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


Processing frame 0
Processing frame 100
Processing frame 200
Processing frame 300
Processing frame 400
Processing frame 500
Processing frame 600
Processing frame 700
Processing frame 800
Processing frame 900
Processing frame 1000
Processing frame 1100
Processing frame 1200
Processing frame 1300
Processing frame 1400
Processing frame 1500
Processing frame 1600
Processing frame 1700
Processing frame 1800
Processing frame 1900
Processing frame 2000
